# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
C. Clontz  ->  C. Clontz  |  ['C. Clontz']
E. Nasedkin  ->  E. Nasedkin  |  ['E. Nasedkin']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
S. Li  ->  S. Li  |  ['S. Li']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 51 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2404.03719


extracting tarball to tmp_2404.03719...

 done.
Retrieving document from  https://arxiv.org/e-print/2404.03722


extracting tarball to tmp_2404.03722...

...

S. Li  ->  S. Li  |  ['S. Li']


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure fflux_comp_method_v2.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure fflux_comp_method_v2.pdf as tmp_2404.03858/./fflux_comp_method_v2.pdf.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 106 bibliographic references in tmp_2404.03858/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2404.03866


extracting tarball to tmp_2404.03866...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.03722-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.03722) | **oMEGACat II -- Photometry and proper motions for 1.4 million stars in  Omega Centauri and its rotation in the plane of the sky**  |
|| <mark>M. Häberle</mark>, et al. -- incl., <mark>N. Neumayer</mark>, <mark>C. Clontz</mark> |
|*Appeared on*| *2024-04-08*|
|*Comments*| *38 pages, 24 figures, 8 tables, Submitted to ApJ, The full catalog will be made public along with the publication by the journal*|
|**Abstract**| Omega Centauri ($\omega$ Cen) is the most massive globular cluster of the Milky Way. It is thought to be the nucleus of an accreted dwarf galaxy because of its high mass and its complex stellar populations. To decipher its formation history and study its dynamics, we created the most comprehensive kinematic catalog for its inner region, by analyzing both archival and new Hubble Space Telescope (HST) data. Our catalog contains 1 395 781 proper-motion measurements out to the half-light radius of the cluster ($\sim$5.0') and down to $m_{F625W}\approx$25. The typical baseline for our proper-motion measurements is 20 years, leading to a median 1D proper motion precision of $\sim$11 $\mu$as yr$^{-1}$ for stars with $m_{F625W}\approx$18 mag, with even better precision ($\sim$6.6 $\mu$as yr$^{-1}$) achieved in the extensively observed centermost (r$<$1.5') region. In addition to our astrometric measurements, we also obtained precise HST photometry in seven filters spanning from the ultraviolet to the near-infrared. This allows detailed color-magnitude-diagram studies and to separate the multiple stellar populations of the cluster. In this work, we describe the data reduction used to obtain both the photometric and the proper-motion measurements. We also illustrate the creation and the content of our catalog, which is made publicly available. Finally, we present measurements of the plane-of-sky rotation of $\omega$ Cen in the previously unprobed inner few arcminutes and a precise measurement of the inclination $i = (43.6\pm1.5)^\circ$. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.03776-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.03776) | **Four-of-a-kind? Comprehensive atmospheric characterisation of the HR  8799 planets with VLTI/GRAVITY**  |
|| <mark>E. Nasedkin</mark>, et al. -- incl., <mark>P. Mollière</mark>, <mark>L. Kreidberg</mark>, <mark>P. Garcia</mark> |
|*Appeared on*| *2024-04-08*|
|*Comments*| *45 pages, 25 figures, submitted to Astronomy & Astrophysics*|
|**Abstract**| With four companions at separations from 16 to 71 au, HR 8799 is a unique target for direct imaging, presenting an opportunity for the comparative study of exoplanets with a shared formation history. Combining new VLTI/GRAVITY observations obtained within the ExoGRAVITY program with archival data, we perform a systematic atmospheric characterisation of all four planets. We explore different levels of model flexibility to understand the temperature structure, chemistry and clouds of each planet using both petitRADTRANS atmospheric retrievals and fits to self-consistent radiative-convective equilibrium models. Using Bayesian Model Averaging to combine multiple retrievals, we find that the HR 8799 planets are highly enriched in metals, with [M/H] $\gtrsim$1, and have stellar to super-stellar C/O ratios. The C/O ratio increases with increasing separation from $0.55^{+0.12}_{-0.10}$ for d to $0.78^{+0.03}_{-0.04}$ for b, with the exception of the innermost planet which has a C/O ratio of $0.87\pm0.03$. By retrieving a quench pressure and using a disequilibrium chemistry model we derive vertical mixing strengths compatible with predictions for high-metallicity, self-luminous atmospheres. Bayesian evidence comparisons strongly favour the presence of HCN in HR 8799 c and e, as well as CH$_{4}$ in HR 8799 c, with detections at $>5\sigma$ confidence. All of the planets are cloudy, with no evidence for patchiness. The clouds of c, d and e are best fit by silicate clouds lying above a deep iron cloud layer, while the clouds of the cooler HR 8799 b are more likely composed of Na$_{2}$S. With well defined atmospheric properties, future exploration of this system is well positioned to unveil further detail in these planets, extending our understanding of the composition, structure, and formation history of these siblings. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.03858-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.03858) | **INvestigations of massive Filaments ANd sTar formation (INFANT). I. Core  Identification and Core Mass Function**  |
|| Y. Cheng, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-04-08*|
|*Comments*| *25 pages, 8 figures, accepted for ApJ*|
|**Abstract**| Filamentary structures are ubiquitously found in high-mass star-forming clouds. To investigate the relationship between filaments and star formation, we carry out the INFANT (INvestigations of massive Filaments ANd sTar formation) survey, a multi-scale, multi-wavelength survey of massive filamentary clouds with ALMA band 3/band 6 and VLA K band. In this first paper, we present the ALMA band 6 continuum observations toward a sample of 8 high-mass star forming filaments. We covered each target with approximately rectangular mosaic field of view with two 12-m array configurations, achieving an angular resolution of $\sim$0.6" (2700 AU at 4.5 kpc) and a continuum rms of $\sim$0.1 mJy/beam ($\sim$0.06 Msun in gas mass assuming 15 K). We identify cores using the getsf and astrodendro and find the former is more robust in terms of both identification and measuring flux densities. We identify in total 183 dense cores (15--36 cores in each cloud) and classify their star formation states via outflow and warm gas tracers. The protostellar cores are statistically more massive than the prestellar cores, possibly indicating further accretion onto cores after formation of protostars. For the high-mass end ($M_\text{core}$ $>$ 1.5 Msun) of the core mass function (CMF) we derive a power-law index of $-$1.15 $\pm$ 0.12 for the whole sample, and $-$1.70 $\pm$ 0.25 for the prestellar population. We also find a steepening trend in CMF with cloud evolution ($-$0.89 $\pm$ 0.15 for the young group v.s. $-$1.44 $\pm$ 0.25 for the evolved group) and discuss its implication for cluster formation. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.03719-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.03719) | **The black hole low mass X-ray binary V404 Cygni is part of a wide  hierarchical triple, and formed without a kick**  |
|| K. B. Burdge, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-04-08*|
|*Comments*| *36 pages, 10 figures, 1 table*|
|**Abstract**| Evidence suggests that when compact objects such as black holes and neutron stars form, they may receive a ``natal kick,'' where the stellar remnant gains momentum. Observational evidence for neutron star kicks is substantial, yet limited for black hole natal kicks, and some proposed black hole formation scenarios result in very small kicks. Here, we report the discovery that the canonical black hole low-mass X-ray binary V404 Cygni is part of a wide hierarchical triple with a tertiary companion at least 3500 astronomical units away from the inner binary. Given the orbital configuration, the black hole likely received a sub-5 kilometer per second kick to have avoided unbinding the tertiary. This discovery reveals that at least some black holes form with nearly no natal kick. Furthermore, the tertiary in this system lends credence to evolutionary models of low-mass X-ray binaries involving a hierarchical triple structure. Remarkably, the tertiary is evolved, indicating that the system formed 3-5 billion years ago, and that the black hole has removed at least half a solar mass of matter from its evolved secondary companion. During the event in which the black hole formed, it is likely that at least half of the mass of the black hole progenitor collapsed into the black hole; it may even have undergone a complete implosion, enabling the tertiary to remain loosely bound. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.03866-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.03866) | **Derivative Spectroscopy and its Application at Detecting the Weak  Emission/Absorption Lines**  |
|| L. Yu, et al. -- incl., <mark>J. Li</mark>, <mark>J. Li</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2024-04-08*|
|*Comments*| **|
|**Abstract**| The development of spectroscopic survey telescopes like Large Sky Area Multi-Object Fiber Spectroscopic Telescope (LAMOST), Apache Point Observatory Galactic Evolution Experiment and Sloan Digital Sky Survey has opened up unprecedented opportunities for stellar classification. Specific types of stars, such as early-type emission-line stars and those with stellar winds, can be distinguished by the profiles of their spectral lines. In this paper, we introduce a method based on derivative spectroscopy (DS) designed to detect signals within complex backgrounds and provide a preliminary estimation of curve profiles. This method exhibits a unique advantage in identifying weak signals and unusual spectral line profiles when compared to other popular line detection methods. We validated our approach using synthesis spectra, demonstrating that DS can detect emission signals three times fainter than Gaussian fitting. Furthermore, we applied our method to 579,680 co-added spectra from LAMOST Medium-Resolution Spectroscopic Survey, identifying 16,629 spectra with emission peaks around the H{\alpha} line from 10,963 stars. These spectra were classified into three distinct morphological groups, resulting in nine subclasses as follows. (1) Emission peak above the pseudo-continuum line (single peak, double peaks, emission peak situated within an absorption line, P Cygni profile, Inverse P Cygni profile); (2) Emission peak below the pseudo-continuum line (sharp emission peak, double absorption peaks, emission peak shifted to one side of the absorption line); (3) Emission peak between the pseudo-continuum line. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2404.03722.md
    + _build/html/tmp_2404.03722/./comparison_with_bellini_uncorrected.jpg
    + _build/html/tmp_2404.03722/./vpd_and_cmd.jpg
    + _build/html/tmp_2404.03722/./gaia_footprint_and_cmd.jpg
exported in  _build/html/2404.03776.md
    + _build/html/tmp_2404.03776/./figures/HR8799bcd_PredictedMRS_gridfits_v04.png
    + _build/html/tmp_2404.03776/./figures/HR8799_AllPlanet_GROUPAintB_KDEHists_v8.png
    + _build/html/tmp_2404.03776/./figures/HR8799e_ParticleR_g.png
exported in  _build/html/2404.03858.md
    + _build/html/tmp_2404.03858/./fmap_iden.png
    + _build/html/tmp_2404.03858/./fcomp_mass_update.png
    + _build/html/tmp_2404.03858/./fcmf_combine.getsf.3panel.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# oMEGACat II - Photometry and proper motions for 1.4 million stars in Omega Centauri and its rotation in the plane of the sky

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.03722-b31b1b.svg)](https://arxiv.org/abs/2404.03722)<mark>Appeared on: 2024-04-08</mark> -  _38 pages, 24 figures, 8 tables, Submitted to ApJ, The full catalog will be made public along with the publication by the journal_

</div>
<div id="authors">

<mark>M. Häberle</mark>, et al. -- incl., <mark>N. Neumayer</mark>, <mark>C. Clontz</mark>

</div>
<div id="abstract">

**Abstract:** Omega Centauri ( $\omc$ ) is the most massive globular cluster of the Milky Way. It is thought to be the nucleus of an accreted dwarf galaxy because of its high mass and its complex stellar populations. To decipher its formation history and study its dynamics, we created the most comprehensive kinematic catalog for its inner region, by analyzing both archival and new _Hubble Space Telescope (HST)_ data. Our catalog contains 1 395 781 proper-motion measurements out to the half-light radius of the cluster ( $\sim5.0\arcmin$ ) and down to $m_{\rm F625W}\approx25$ . The typical baseline for our proper-motion measurements is 20 years, leading to a median 1D proper motion precision of $\sim$ 11 µas yr $^{-1}$ for stars with $m_{\rm F625W}\approx 18$ mag, with even better precision ( $\sim$ 6.6 µas yr $^{-1}$ ) achieved in the extensively observed centermost ( $r<1.5\arcmin$ ) region.  In addition to our astrometric measurements, we also obtained precise _HST_ photometry in seven filters spanning from the ultraviolet to the near-infrared. This allows detailed color-magnitude-diagram studies and to separate the multiple stellar populations of the cluster. In this work, we describe the data reduction used to obtain both the photometric and the proper-motion measurements. We also illustrate the creation and the content of our catalog, which is made publicly available. Finally, we present measurements of the plane-of-sky rotation of $\omc$ in the previously unprobed inner few arcminutes and a precise measurement of the inclination $i=(43.6\pm1.5)^\circ$ .

</div>

<div id="div_fig1">

<img src="tmp_2404.03722/./comparison_with_bellini_uncorrected.jpg" alt="Fig20" width="100%"/>

**Figure 20. -** Comparison of uncorrected proper motions with [Bellini, Anderson and Bedin (2017)]() for the right-ascension (upper row) and the declination proper-motion component (lower row). The panels first-from-the-left show a direct comparison of the proper-motion components. The red line is the plane-bisector and not a fit to the data. The second-from-the-left panels show how the proper motion difference between the two datasets varies over the field. Some mild systematic trends are visible, as expected for the uncorrected, amplifier-based, proper motions. The third-from-the-left panels show a histogram of the differences between the measurements, the fourth-from-the-left panels show the same differences but divided by the combined proper-motion error. (*fig:bellini_uncorrected*)

</div>
<div id="div_fig2">

<img src="tmp_2404.03722/./vpd_and_cmd.jpg" alt="Fig7" width="100%"/>

**Figure 7. -** **Left panel: ** A vector-point diagram of the relative proper motions for a subset of around 700 000 well-measured stars in our catalog. Most stars are distributed around the origin, as expected for \omc's member stars and follow a normal distribution in both velocity components (see marginalized histograms at the edge of the plot). A small fraction of stars has a relative proper motion incompatible with the cluster's motion. Those stars lie outside of the 3.0 mas yr$^{-1}$ radius indicated with a dashed circle (corresponding to a 4.5$\sigma$ deviation from the velocity distribution of the cluster stars) and are marked with turquoise crosses. The non-member stars show substructures that can be attributed to background galaxies (marked with a solid circle) and Galactic foreground/background stars. **Right panel:** A color-magnitude diagram of the same sample. One can see that the stars with a high relative proper motion do not follow the CMD sequences of \omc. Our proper motions cover almost the entire CMD, however the brightest stars, indicated with red dots, do not have a proper motion measurement, as they are saturated even in the shortest _HST_ exposures. (*fig:vpd*)

</div>
<div id="div_fig3">

<img src="tmp_2404.03722/./gaia_footprint_and_cmd.jpg" alt="Fig13" width="100%"/>

**Figure 13. -** **Left panel:** Footprint of our proper-motion catalog and all _Gaia_ stars that could be crossmatched (blue). High-quality stars from _Gaia FPR_ are marked in cyan, and high-quality stars from _Gaia DR3_ are marked in red. **Middle panel:** The same sample of stars, but plotted as a color-magnitude diagram. One can see that all HQ _Gaia DR3_ measurements are limited to stars brighter than the main-sequence turn-off, while the _Gaia FPR_ sample reaches slightly deeper. **Right panel:** Absolute deviation between the positions from our _HST_ catalog and the two HQ _Gaia_ subsamples. The shift of the centroid can be explained by the absolute proper motion of \omc , while the spread of the distribution is caused by the displacement due to the velocity dispersion. (*fig:gaia_footprint*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.03722"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\klip}{{\tt KLIP}\xspace}$
$\newcommand{\pyn}{{\tt PynPoint}\xspace}$
$\newcommand{\kgold}{K_{\rm Gold}}$
$\newcommand{\kliss}{K_{\rm Liss}}$
$\newcommand{\bliss}{B_{\rm Liss}}$
$\newcommand{\rcapt}{R_{\rm capt}}$
$\newcommand{\mz}{M_{\rm Z}}$
$\newcommand{\mxy}{M_{\rm XY}}$
$\newcommand{\lext}{L_{\rm ext}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\fpgsun}{f_{\rm D/G,\odot}}$
$\newcommand{\mdisc}{M_{\rm disc}}$
$\newcommand{\tdisc}{t_{\rm disc}}$
$\newcommand{\mdotz}{\dot{M}_{\rm Z}}$
$\newcommand{\mdotxy}{\dot{M}_{\rm XY}}$
$\newcommand{\cadia}{C_{\rm adia }}$
$\newcommand{\cloc}{C_{\rm loc }}$
$\newcommand{\fvisc}{f_{\rm visc }}$
$\newcommand{\fsigma}{f_{\rm \Sigma}}$
$\newcommand{\xs}{x_{\rm s}}$
$\newcommand{\rp}{r_{\rm p}}$
$\newcommand{\omegap}{\Omega_{\rm p}}$
$\newcommand{\sigmap}{\Sigma_{\rm p}}$
$\newcommand{\tauuturn}{\tau_{\rm U-turn}}$
$\newcommand{\taucool}{\tau_{\rm cool}}$
$\newcommand{\taulib}{\tau_{\rm lib}}$
$\newcommand{\tauvisc}{\tau_{\rm visc}}$
$\newcommand{\msat}{M_{\rm sat}}$
$\newcommand{\mgap}{M_{\rm gap}}$
$\newcommand{\rroche}{r_{\rm Roche}}$
$\newcommand{\betat}{\beta_{\rm T}}$
$\newcommand{\betas}{\beta_{\rm \Sigma}}$
$\newcommand{\dotrp}{\dot{r}_{\rm p}}$
$\newcommand{\rch}[1]{{{\color{orange}#1}}}$
$\newcommand{\rea}[1]{{{\color{red}Reading: #1}}}$
$\newcommand{\cstart}$
$\newcommand{\cend}$
$\newcommand{\checkmark}{\tikz\fill[scale=0.4](0,.35) -- (.25,0) -- (1,.7) -- (.25,.15) -- cycle;}$
$\newcommand{\ptrad}{{\tt petitRADTRANS}\xspace}$
$\newcommand{\ptdoc}{\url{https://petitradtrans.readthedocs.io}}$
$\newcommand{\mearth}{{\rm M}_\oplus}$
$\newcommand{\rearth}{{\rm R}_\oplus}$
$\newcommand{\ed}{{\sc{EddySed}}\xspace}$
$\newcommand{\msun}{{\rm M}_\odot}$
$\newcommand{\rsun}{{\rm R}_\odot}$
$\newcommand{\rhill}{R_{\rm H}}$
$\newcommand{\mcore}{M_{\rm core}}$
$\newcommand{\mheavy}{M_{\rm heavy}}$
$\newcommand{\astart}{a_{\rm start}}$
$\newcommand{\fpg}{f_{\rm D/G}}$
$\newcommand{\fri}{f_{\rm R/I}}$
$\newcommand{\f1}{f_{\rm I}}$
$\newcommand{\mj}{{M}_{\textrm{\tiny Jup}}\xspace}$
$\newcommand{\rj}{{R}_{\textrm{\tiny Jup}}\xspace}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\Tint}{\ensuremath{T_{\rm int}}\xspace}$
$\newcommand{\Tbot}{\ensuremath{T_{\rm bot}}\xspace}$
$\newcommand{\kzz}{\ensuremath{K_{\rm zz}}\xspace}$
$\newcommand{\fsed}{\ensuremath{f_{\rm sed}}\xspace}$
$\newcommand{\h2o}{H_{2}O\xspace}$
$\newcommand{\ch4}{CH_{4}\xspace}$
$\newcommand{\nh3}{NH_{3}\xspace}$
$\newcommand{\co2}{CO_{2}\xspace}$
$\newcommand{\ph3}{PH_{3}\xspace}$
$\newcommand{\Zpl}{\ensuremath{Z_{\rm pl}}\xspace}$
$\newcommand{\Zstar}{\ensuremath{Z_{\star}}\xspace}$
$\newcommand{\mstar}{M_*}$
$\newcommand{\mdisk}{M_{\rm disk}}$
$\newcommand{\astart}{a_{\rm start}}$
$\newcommand{\tstart}{t_{\rm start}}$
$\newcommand{\membstart}{M_{\rm emb,0}}$
$\newcommand{\memb}{M_{\rm emb}}$
$\newcommand{\anorm}{a_0}$
$\newcommand{\rcore}{R_{\rm core}}$
$\newcommand{\mdotcore}{\dot{M}_{\rm core}}$
$\newcommand{\mwind}{\dot{M}_{\rm w}}$
$\newcommand{\menv}{M_{\rm env}}$
$\newcommand{\miso}{M_{\rm iso}}$
$\newcommand{\mpla}{m_{\rm pla}}$
$\newcommand{\arock}{a_{\rm rock}}$
$\newcommand{\aice}{a_{\rm ice}}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\mplanet}{M_{\rm planet}}$
$\newcommand{\aplanet}{a_{\rm planet}}$
$\newcommand{\sigmad}{\Sigma_{\rm D}}$
$\newcommand{\atouch}{a_{\rm touch}}$
$\newcommand{\sigmanorm}{\Sigma_{0}}$
$\newcommand{\tmc}{\tau_{\rm RV,MC}}$
$\newcommand{\epsilonmc}{\epsilon_{\rm RV,MC}}$
$\newcommand{\nref}{N_{\rm ref}}$
$\newcommand{\nobsref}{N_{\rm obsref}}$
$\newcommand{\nhot}{N_{\rm hot}}$
$\newcommand{\nlate}{N_{\rm late}}$
$\newcommand{\nobs}{N_{\rm obs}}$
$\newcommand{\nsamp}{N_{\rm samp}}$
$\newcommand{\dks}{d_{\rm KS}}$
$\newcommand{\sam}{S_{\rm a-M}}$
$\newcommand{\sm}{S_{\rm M}}$
$\newcommand{\sa}{S_{\rm a}}$
$\newcommand{\sfeh}{S_{\rm[Fe/H]}}$
$\newcommand{\nobsreffv}{N_{\rm obsrefFV}}$
$\newcommand{\nobsrefhotfv}{N_{\rm obsrefhotFV}}$
$\newcommand{\msini}{M\sin i}$
$\newcommand{\amax}{a_{\rm max}}$
$\newcommand{\amin}{a_{\rm min}}$
$\newcommand{\aphys}{a_{\rm phys}}$
$\newcommand{\pwhot}{P_{\rm whot}}$
$\newcommand{\pfv}{P_{\rm FV}}$
$\newcommand{\pfvwhot}{P_{\rm FV,whot}}$
$\newcommand{\ffv}{F_{\rm FV}}$
$\newcommand{\fwemb}{F_{\rm wemb}}$
$\newcommand{\mmax}{M_{\rm max}}$
$\newcommand{\fopa}{f_{\rm opa}}$
$\newcommand{\t2}{\tau_{\rm II}}$
$\newcommand{\ltot}{L_{\rm tot}}$
$\newcommand{\lcore}{L_{\rm core}}$
$\newcommand{\mtot}{M_{\rm tot}}$
$\newcommand{\sigmas0}{\Sigma_{\rm s,0}}$
$\newcommand{\LJ}{L_{{\rm \jupiter}}}$
$\newcommand{\RJ}{R_{{\rm \jupiter}}}$
$\newcommand{\petit}{\emph{petitCODE} }$
$\newcommand{\kgold}{K_{\rm Gold}}$
$\newcommand{\kliss}{K_{\rm Liss}}$
$\newcommand{\bliss}{B_{\rm Liss}}$
$\newcommand{\rcapt}{R_{\rm capt}}$
$\newcommand{\mz}{M_{\rm Z}}$
$\newcommand{\mxy}{M_{\rm XY}}$
$\newcommand{\lext}{L_{\rm ext}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\fpgsun}{f_{\rm D/G,\odot}}$
$\newcommand{\mdisc}{M_{\rm disc}}$
$\newcommand{\tdisc}{t_{\rm disc}}$
$\newcommand{\mdotz}{\dot{M}_{\rm Z}}$
$\newcommand{\mdotxy}{\dot{M}_{\rm XY}}$
$\newcommand{\cadia}{C_{\rm adia }}$
$\newcommand{\cloc}{C_{\rm loc }}$
$\newcommand{\fvisc}{f_{\rm visc }}$
$\newcommand{\fsigma}{f_{\rm \Sigma}}$
$\newcommand{\xs}{x_{\rm s}}$
$\newcommand{\rp}{r_{\rm p}}$
$\newcommand{\omegap}{\Omega_{\rm p}}$
$\newcommand{\sigmap}{\Sigma_{\rm p}}$
$\newcommand{\tauuturn}{\tau_{\rm U-turn}}$
$\newcommand{\taucool}{\tau_{\rm cool}}$
$\newcommand{\taulib}{\tau_{\rm lib}}$
$\newcommand{\tauvisc}{\tau_{\rm visc}}$
$\newcommand{\msat}{M_{\rm sat}}$
$\newcommand{\mgap}{M_{\rm gap}}$
$\newcommand{\rroche}{r_{\rm Roche}}$
$\newcommand{\betat}{\beta_{\rm T}}$
$\newcommand{\betas}{\beta_{\rm \Sigma}}$
$\newcommand{\dotrp}{\dot{r}_{\rm p}}$
$\newcommand\vol{#1}{ #1}$
$\newcommand{\(}{\left(}$
$\newcommand{\)}{\right)}$
$\newcommand{\<}{\left<}$
$\newcommand{\>}{\right>}$
$\newcommand{\bacon}{\emph{BACON}~}$
$\newcommand{\tr}{\mathcal{T}}$
$\newcommand{\rch}[1]{{{\color{orange}#1}}}$
$\newcommand{\rea}[1]{{{\color{red}Reading: #1}}}$
$\newcommand{\cstart} $
$\newcommand{\cend} $
$\newcommandcitealias{ackerman_precipitating_2001}{AM01}$
$\newcommandcitealias{molliere_retrieving_2020}{M20}$
$\newcommandcitealias{zhang_elemental_2023}{Z23}$
$\newcommandcitealias{guillot_radiative_2010}{G10}$
$\begin{document}$
$   \title{Four-of-a-kind? Comprehensive atmospheric characterisation of the HR~8799 planets with VLTI/GRAVITY}$
$    \titlerunning{HR8799 Atmospheres}$
$    \authorrunning{Nasedkin et al.}$
$   \subtitle $
$   \author{E.~Nasedkin\inst{\ref{mpia}}\fnmsep\thanks{Corresponding Author, \href{mailto:nasedkin@mpia.de}{nasedkin@mpia.de}}$
$   \and P.~Mollière\inst{\ref{mpia}}$
$ \and S.~Lacour\inst{\ref{lesia},\ref{esog}}$
$ \and M.~Nowak\inst{\ref{cam}}$
$ \and L.~Kreidberg\inst{\ref{mpia}}$
$ \and T.~Stolker\inst{\ref{leiden}}$
$ \and J.~J.~Wang\inst{\ref{northwestern}}$
$ \and W.~O.~Balmer\inst{\ref{jhupa},\ref{stsci}}$
$ \and J.~Kammerer\inst{\ref{esog}}$
$ \and J.~Shangguan\inst{\ref{mpe}}$
$ \and R.~Abuter\inst{\ref{esog}}$
$ \and A.~Amorim\inst{\ref{lisboa},\ref{centra}}$
$ \and R.~Asensio-Torres\inst{\ref{mpia}}$
$ \and M.~Benisty\inst{\ref{ipag}}$
$ \and J.-P.~Berger\inst{\ref{ipag}}$
$ \and H.~Beust\inst{\ref{ipag}}$
$ \and S.~Blunt\inst{\ref{northwestern}}$
$ \and A.~Boccaletti\inst{\ref{lesia}}$
$ \and M.~Bonnefoy\inst{\ref{ipag}}$
$ \and H.~Bonnet\inst{\ref{esog}}$
$ \and M.~S.~Bordoni\inst{\ref{mpe}}$
$ \and G.~Bourdarot\inst{\ref{mpe}}$
$ \and W.~Brandner\inst{\ref{mpia}}$
$ \and F.~Cantalloube\inst{\ref{lam}}$
$ \and P.~Caselli \inst{\ref{mpe}}$
$ \and B.~Charnay\inst{\ref{lesia}}$
$ \and G.~Chauvin\inst{\ref{cotedazur}}$
$ \and A.~Chavez\inst{\ref{northwestern}}$
$ \and E.~Choquet\inst{\ref{lam}}$
$ \and V.~Christiaens\inst{\ref{liege}}$
$ \and Y.~Clénet\inst{\ref{lesia}}$
$ \and V.~Coudé~du~Foresto\inst{\ref{lesia}}$
$ \and A.~Cridland\inst{\ref{leiden}}$
$ \and R.~Davies\inst{\ref{mpe}}$
$ \and R.~Dembet\inst{\ref{lesia}}$
$ \and J.~Dexter\inst{\ref{boulder}}$
$ \and A.~Drescher\inst{\ref{mpe}}$
$ \and G.~Duvert\inst{\ref{ipag}}$
$ \and A.~Eckart\inst{\ref{cologne},\ref{bonn}}$
$ \and F.~Eisenhauer\inst{\ref{mpe}}$
$ \and N.~M.~Förster Schreiber\inst{\ref{mpe}}$
$ \and P.~Garcia\inst{\ref{centra},\ref{porto}}$
$ \and R.~Garcia~Lopez\inst{\ref{dublin},\ref{mpia}}$
$ \and E.~Gendron\inst{\ref{lesia}}$
$ \and R.~Genzel\inst{\ref{mpe},\ref{ucb}}$
$ \and S.~Gillessen\inst{\ref{mpe}}$
$ \and J.~H.~Girard\inst{\ref{stsci}}$
$ \and S.~Grant\inst{\ref{mpe}}$
$ \and X.~Haubois\inst{\ref{esoc}}$
$ \and G.~Hei\ss el\inst{\ref{actesa},\ref{lesia}}$
$ \and Th.~Henning\inst{\ref{mpia}}$
$ \and S.~Hinkley\inst{\ref{exeter}}$
$ \and S.~Hippler\inst{\ref{mpia}}$
$ \and M.~Houllé\inst{\ref{cotedazur}}$
$ \and Z.~Hubert\inst{\ref{ipag}}$
$ \and L.~Jocou\inst{\ref{ipag}}$
$ \and M.~Keppler\inst{\ref{arizona}}$
$ \and P.~Kervella\inst{\ref{lesia}}$
$ \and N.~T.~Kurtovic\inst{\ref{mpe}}$
$ \and A.-M.~Lagrange\inst{\ref{ipag},\ref{lesia}}$
$ \and V.~Lapeyrère\inst{\ref{lesia}}$
$ \and J.-B.~Le~Bouquin\inst{\ref{ipag}}$
$ \and D.~Lutz\inst{\ref{mpe}}$
$ \and A.-L.~Maire\inst{\ref{ipag}}$
$ \and F.~Mang\inst{\ref{mpe}}$
$ \and G.-D.~Marleau\inst{\ref{duisburg},\ref{tuebingen},\ref{bern},\ref{mpia}}$
$ \and A.~Mérand\inst{\ref{esog}}$
$ \and J.~D.~Monnier\inst{\ref{umich}}$
$ \and C.~Mordasini\inst{\ref{bern}}$
$ \and T.~Ott\inst{\ref{mpe}}$
$ \and G.~P.~P.~L.~Otten\inst{\ref{sinica}}$
$ \and C.~Paladini\inst{\ref{esoc}}$
$ \and T.~Paumard\inst{\ref{lesia}}$
$ \and K.~Perraut\inst{\ref{ipag}}$
$ \and G.~Perrin\inst{\ref{lesia}}$
$ \and O.~Pfuhl\inst{\ref{esog}}$
$ \and N.~Pourré\inst{\ref{ipag}}$
$ \and L.~Pueyo\inst{\ref{stsci}}$
$ \and D.~C.~Ribeiro\inst{\ref{mpe}}$
$ \and E.~Rickman\inst{\ref{esa}}$
$ \and J.B.~Ruffio\inst{\ref{sandiego}}$
$ \and Z.~Rustamkulov\inst{\ref{jhueps}}$
$ \and T.~Shimizu \inst{\ref{mpe}}$
$ \and D.~Sing\inst{\ref{jhupa},\ref{jhueps}}$
$ \and J.~Stadler\inst{\ref{mpa},\ref{origins}}$
$ \and O.~Straub\inst{\ref{origins}}$
$ \and C.~Straubmeier\inst{\ref{cologne}}$
$ \and E.~Sturm\inst{\ref{mpe}}$
$ \and L.~J.~Tacconi\inst{\ref{mpe}}$
$ \and E.~F.~van~Dishoeck\inst{\ref{leiden},\ref{mpe}}$
$ \and A.~Vigan\inst{\ref{lam}}$
$ \and F.~Vincent\inst{\ref{lesia}}$
$ \and S.~D.~von~Fellenberg\inst{\ref{bonn}}$
$ \and F.~Widmann\inst{\ref{mpe}}$
$ \and T.~O.~Winterhalder\inst{\ref{esog}}$
$ \and J.~Woillez\inst{\ref{esog}}$
$ \and \c{S}.~Yaz\i c\i \inst{\ref{mpe}}$
$ \and  the~GRAVITY~Collaboration}$
$\institute{$
$   Max-Planck-Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany$
$\label{mpia}      \and$
$   LESIA, Observatoire de Paris, PSL, CNRS, Sorbonne Université, Université de Paris, 5 place Janssen, 92195 Meudon, France$
$\label{lesia}      \and$
$   European Southern Observatory, Karl-Schwarzschild-Stra\ss e 2, 85748 Garching, Germany$
$\label{esog}      \and$
$   Institute of Astronomy, University of Cambridge, Madingley Road, Cambridge CB3 0HA, United Kingdom$
$\label{cam}      \and$
$   Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands$
$\label{leiden}      \and$
$   Center for Interdisciplinary Exploration and Research in Astrophysics (CIERA) and Department of Physics and Astronomy, Northwestern University, Evanston, IL 60208, USA$
$\label{northwestern}      \and$
$   Department of Physics \& Astronomy, Johns Hopkins University, 3400 N. Charles Street, Baltimore, MD 21218, USA$
$\label{jhupa}      \and$
$   Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA$
$ \label{stsci}  \and$
$   Max-Planck-Institut für Extraterrestrische Physik, Giessenbachstra\ss e~1, 85748 Garching, Germany$
$\label{mpe}      \and$
$   Universidade de Lisboa - Faculdade de Ci\^encias, Campo Grande, 1749-016 Lisboa, Portugal$
$\label{lisboa}      \and$
$   CENTRA - Centro de Astrofísica e Gravitação, IST, Universidade de Lisboa, 1049-001 Lisboa, Portugal$
$\label{centra}      \and$
$   Univ. Grenoble Alpes, CNRS, IPAG, 38000 Grenoble, France$
$\label{ipag}      \and$
$   Aix Marseille Univ, CNRS, CNES, LAM, Marseille, France$
$\label{lam}      \and$
$   Université C\^{o}te d'Azur, Observatoire de la C\^{o}te d'Azur, CNRS, Laboratoire Lagrange, France$
$\label{cotedazur}      \and$
$  STAR Institute, Université de Liège, Allée du Six Ao\^ut 19c, 4000 Liège, Belgium$
$\label{liege}      \and$
$   Department of Astrophysical \& Planetary Sciences, JILA, Duane Physics Bldg., 2000 Colorado Ave, University of Colorado, Boulder, CO 80309, USA$
$\label{boulder}      \and$
$   1. Institute of Physics, University of Cologne, Zülpicher Stra\ss e 77, 50937 Cologne, Germany$
$\label{cologne}      \and$
$   Max-Planck-Institut für Radioastronomie, Auf dem Hügel 69, 53121 Bonn, Germany$
$\label{bonn}      \and$
$   Universidade do Porto, Faculdade de Engenharia, Rua Dr.~Roberto Frias, 4200-465 Porto, Portugal$
$\label{porto}      \and$
$   School of Physics, University College Dublin, Belfield, Dublin 4, Ireland$
$\label{dublin}      \and$
$   Astrophysics Group, Department of Physics \& Astronomy, University of Exeter, Stocker Road, Exeter, EX4 4QL, United Kingdom$
$\label{exeterAstro}      \and$
$   Departments of Physics and Astronomy, Le Conte Hall, University of California, Berkeley, CA 94720, USA$
$\label{ucb}      \and$
$   European Southern Observatory, Casilla 19001, Santiago 19, Chile$
$\label{esoc}      \and$
$   Advanced Concepts Team, European Space Agency, TEC-SF, ESTEC, Keplerlaan 1, NL-2201, AZ Noordwijk, The Netherlands$
$\label{actesa}      \and$
$Department of Astronomy and Steward Observatory, University of Arizona, Tucson, AZ, USA$
$\label{arizona}     \and$
$   University of Exeter, Physics Building, Stocker Road, Exeter EX4 4QL, United Kingdom$
$\label{exeter}      \and$
$   Fakultät für Physik, Universität Duisburg--Essen, Lotharstra\ss e 1, 47057 Duisburg, Germany$
$\label{duisburg}      \and$
$   Institüt für Astronomie und Astrophysik, Universität Tübingen, Auf der Morgenstelle 10, 72076 Tübingen, Germany$
$\label{tuebingen}      \and$
$   Physikalisches Institut, Universität Bern, Gesellschaftsstr.~6, 3012 Bern, Switzerland$
$\label{bern}      \and$
$   Astronomy Department, University of Michigan, Ann Arbor, MI 48109 USA$
$\label{umich}      \and$
$   Academia Sinica, Institute of Astronomy and Astrophysics, 11F Astronomy-Mathematics Building, NTU/AS campus, No. 1, Section 4, Roosevelt Rd., Taipei 10617, Taiwan$
$\label{sinica}      \and$
$   European Space Agency,$
$   ESA Office, Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA$
$\label{esa}   \and$
$    Department of Astronomy \& Astrophysics,  University of California, San Diego, La Jolla, CA 92093, USA$
$ \label{sandiego}   \and$
$   Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA$
$\label{jhueps}      \and$
$   Max-Planck-Institut für Astrophysik, Karl-Schwarzschild-Str. 1, 85741 Garching, Germany$
$\label{mpa}      \and$
$   Excellence Cluster ORIGINS, Boltzmannstra\ss e 2, 85748 Garching bei München, Germany$
$\label{origins}      \and$
$   Institutionen för astronomi, AlbaNova universitetscentrum, Stockholms universitet, 10691 Stockholm, Sweden$
$\label{stockholm}$
$}$
$   \date{Received 2024-01-24}$
$  \abstract{$
$  With four companions at separations from 16 to 71 au, HR 8799 is a unique target for direct imaging, presenting an opportunity for the comparative study of exoplanets with a shared formation history.$
$  Combining new VLTI/GRAVITY observations obtained within the ExoGRAVITY program with archival data, we perform a systematic atmospheric characterisation across all four planets.$
$  We explore different levels of model flexibility to understand the temperature structure, chemistry and clouds of each planet using both {\tt petitRADTRANS} atmospheric retrievals and fits to self-consistent radiative-convective equilibrium models.$
$  Using Bayesian Model Averaging to combine multiple retrievals (a total of 89 across all four planets), we find that the HR 8799 planets are highly enriched in metals, with [M/H] \gtrsim1, and have stellar to super-stellar C/O ratios.$
$  The C/O ratio increases with increasing separation from 0.55^{+0.12}_{-0.10} for d to 0.78^{+0.03}_{-0.04} for b, with the exception of the innermost planet which has a C/O ratio of 0.87\pm0.03.$
$  Such high metallicities are unexpected for these massive planets, challenging planet formation models.$
$  By retrieving a quench pressure and using a disequilibrium chemistry model we derive vertical mixing strengths compatible with predictions for high-metallicity, self-luminous atmospheres.$
$  Bayesian evidence comparisons strongly favour the presence of HCN in HR 8799 c and e, as well as \ch4 in HR 8799 c, with detections at >5\sigma confidence.$
$  All of the planets are cloudy, with no evidence for patchiness.$
$  The clouds of c, d and e are best fit by silicate clouds lying above a deep iron cloud layer, while the clouds of the cooler HR 8799 b are more likely composed of Na_{2}S.$
$  With well newcommandined atmospheric properties, future exploration of this system is well positioned to unveil further detail in these planets, extending our understanding of the composition, structure, and formation history of these siblings.}$
$   \keywords{Methods: observational – planets and satellites: atmospheres  – instrumentation: interferometers – radiative transfer}$
$   \maketitle$
$\n\end{document}\end{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\mplanet}{M_{\rm planet}}$
$\newcommand{\aplanet}{a_{\rm planet}}$
$\newcommand{\sigmad}{\Sigma_{\rm D}}$
$\newcommand{\atouch}{a_{\rm touch}}$
$\newcommand{\sigmanorm}{\Sigma_{0}}$
$\newcommand{\tmc}{\tau_{\rm RV,MC}}$
$\newcommand{\epsilonmc}{\epsilon_{\rm RV,MC}}$
$\newcommand{\nref}{N_{\rm ref}}$
$\newcommand{\nobsref}{N_{\rm obsref}}$
$\newcommand{\nhot}{N_{\rm hot}}$
$\newcommand{\nlate}{N_{\rm late}}$
$\newcommand{\nobs}{N_{\rm obs}}$
$\newcommand{\nsamp}{N_{\rm samp}}$
$\newcommand{\dks}{d_{\rm KS}}$
$\newcommand{\sam}{S_{\rm a-M}}$
$\newcommand{\sm}{S_{\rm M}}$
$\newcommand{\sa}{S_{\rm a}}$
$\newcommand{\sfeh}{S_{\rm[Fe/H]}}$
$\newcommand{\nobsreffv}{N_{\rm obsrefFV}}$
$\newcommand{\nobsrefhotfv}{N_{\rm obsrefhotFV}}$
$\newcommand{\msini}{M\sin i}$
$\newcommand{\amax}{a_{\rm max}}$
$\newcommand{\amin}{a_{\rm min}}$
$\newcommand{\aphys}{a_{\rm phys}}$
$\newcommand{\pwhot}{P_{\rm whot}}$
$\newcommand{\pfv}{P_{\rm FV}}$
$\newcommand{\pfvwhot}{P_{\rm FV,whot}}$
$\newcommand{\ffv}{F_{\rm FV}}$
$\newcommand{\fwemb}{F_{\rm wemb}}$
$\newcommand{\mmax}{M_{\rm max}}$
$\newcommand{\fopa}{f_{\rm opa}}$
$\newcommand{\t2}{\tau_{\rm II}}$
$\newcommand{\ltot}{L_{\rm tot}}$
$\newcommand{\lcore}{L_{\rm core}}$
$\newcommand{\mtot}{M_{\rm tot}}$
$\newcommand{\sigmas0}{\Sigma_{\rm s,0}}$
$\newcommand{\LJ}{L_{{\rm \jupiter}}}$
$\newcommand{\RJ}{R_{{\rm \jupiter}}}$
$\newcommand{\petit}{\emph{petitCODE} }$
$\newcommand\vol{#1}$
$\newcommand{\(}{\left(}$
$\newcommand{\)}{\right)}$
$\newcommand{\<}{\left<}$
$\newcommand{\>}{\right>}$
$\newcommand{\bacon}{\emph{BACON}~}$
$\newcommand{\tr}{\mathcal{T}}$</div>



<div id="title">

# Four-of-a-kind? Comprehensive atmospheric characterisation of the HR 8799 planets with VLTI/GRAVITY

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.03776-b31b1b.svg)](https://arxiv.org/abs/2404.03776)<mark>Appeared on: 2024-04-08</mark> -  _45 pages, 25 figures, submitted to Astronomy & Astrophysics_

</div>
<div id="authors">

<mark>E. Nasedkin</mark>, et al. -- incl., <mark>P. Mollière</mark>, <mark>L. Kreidberg</mark>, <mark>P. Garcia</mark>

</div>
<div id="abstract">

**Abstract:** With four companions at separations from 16 to 71 au, HR 8799 is a unique target for direct imaging, presenting an opportunity for the comparative study of exoplanets with a shared formation history.  Combining new VLTI/GRAVITY observations obtained within the ExoGRAVITY program with archival data, we perform a systematic atmospheric characterisation across all four planets.  We explore different levels of model flexibility to understand the temperature structure, chemistry and clouds of each planet using both ${\tt petitRADTRANS}$ atmospheric retrievals and fits to self-consistent radiative-convective equilibrium models.  Using Bayesian Model Averaging to combine multiple retrievals (a total of 89 across all four planets), we find that the HR 8799 planets are highly enriched in metals, with [ M/H ] $\gtrsim$ 1, and have stellar to super-stellar C/O ratios.  The C/O ratio increases with increasing separation from $0.55^{+0.12}_{-0.10}$ for d to $0.78^{+0.03}_{-0.04}$ for b, with the exception of the innermost planet which has a C/O ratio of $0.87\pm0.03$ .  Such high metallicities are unexpected for these massive planets, challenging planet formation models.  By retrieving a quench pressure and using a disequilibrium chemistry model we derive vertical mixing strengths compatible with predictions for high-metallicity, self-luminous atmospheres.  Bayesian evidence comparisons strongly favour the presence of HCN in HR 8799 c and e, as well as $\ch$ 4 in HR 8799 c, with detections at $>5\sigma$ confidence.  All of the planets are cloudy, with no evidence for patchiness.  The clouds of c, d and e are best fit by silicate clouds lying above a deep iron cloud layer, while the clouds of the cooler HR 8799 b are more likely composed of Na $_{2}$ S.  With well defined atmospheric properties, future exploration of this system is well positioned to unveil further detail in these planets, extending our understanding of the composition, structure, and formation history of these siblings.

</div>

<div id="div_fig1">

<img src="tmp_2404.03776/./figures/HR8799bcd_PredictedMRS_gridfits_v04.png" alt="Fig7" width="100%"/>

**Figure 7. -** Predictions for MIRI/MRS based on the most favoured disequilibrium (solid) and free chemistry (dashed) retrievals, together with the best-fit self-consistent models from each grid. (*fig:mrs_predictions*)

</div>
<div id="div_fig2">

<img src="tmp_2404.03776/./figures/HR8799_AllPlanet_GROUPAintB_KDEHists_v8.png" alt="Fig14" width="100%"/>

**Figure 14. -** Bayesian averaged posterior parameter distributions for each of the HR 8799 planets, based on the group $A\cap B$ set of retrievals. In faint lines beneath the total posterior distribution are the individual contributions from different retrievals. The coloured dashed and dotted lines indicate the median and $\pm34.1$\% confidence regions respectively. The vertical grey lines indicate typical parameter values (e.g. solar metallicity and C/O) and serve as a visual reference for each parameter. For the planet mass, they indicate the dynamical mass estimate from \cite{zurlo_orbital_2022}. (*fig:retrieval_parameter_summary*)

</div>
<div id="div_fig3">

<img src="tmp_2404.03776/./figures/HR8799e_ParticleR_g.png" alt="Fig4" width="100%"/>

**Figure 4. -** Effective particle radii as a function of altitude for silicate (blue) and iron (red) clouds. The solid lines indicate the radii for Case e.AB.13, which used a free cloud base pressure and abundance, while the dashed lines are for Case e.AB.2, which used equilibrium condensation and scaled equilibrium abundances. The horizontal lines indicate the cloud base pressure. The green line indicates the temperature profile, with the free cloud profile in dark green and the equilibrium cloud profile in light green. The shaded regions indicate unphysical particle sizes where the opacity contribution is set to 0. (*fig:cloud_particle_radius*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.03776"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{\mbox{km s^{-1}}}$
$\newcommand{\kmspc}{\mbox{km s^{-1} pc^{-1}}}$
$\newcommand{\sqc}{\mbox{cm^{-2}}}$
$\newcommand{ç}{\mbox{cm^{-3}}}$
$\newcommand{\lsol}{\mbox{L_\odot}}$
$\newcommand{\msol}{\mbox{M_\odot}}$
$\newcommand{\msolpyr}{\mbox{M_\odot yr^{-1}}}$
$\newcommand{\mujypbm}{\mbox{\muJy beam^{-1}}}$
$\newcommand{\mjypbm}{\mbox{mJy beam^{-1}}}$
$\newcommand{\jypbm}{\mbox{Jy beam^{-1}}}$
$\newcommand{\hii}{\mbox{H {\sc ii}}}$
$\newcommand{\vlsr}{\mbox{V_\text{lsr}}}$
$\newcommand{\methanol}{\mbox{\rm CH_3OH}}$
$\newcommand{\ntwodp}{\mbox{\rm N_2D^+}}$
$\newcommand{\ntwohp}{\mbox{\rm N_2H^+}}$
$\newcommand{\htwoco}{\mbox{\rm H_2CO}}$
$\newcommand{\ceighteeno}{\mbox{\rm C^{18}O}}$
$\newcommand{\thirteenco}{\mbox{\rm ^{13}CO}}$
$\newcommand{\dcop}{\mbox{\rm DCO^+}}$
$\newcommand{\hcop}{\mbox{\rm HCO^+}}$
$\newcommand{\dcn}{\mbox{\rm DCN}}$
$\newcommand{\ammonia}{\mbox{\rm NH_3}}$
$\newcommand{\gcm}{\mbox{g cm^{-2}}}$
$\newcommand{\lsun}{\mbox{ L_\odot}}$
$\newcommand{\msun}{\mbox{ M_\odot}}$
$\newcommand{\msunyr}{\mbox{ M_\odot yr^{-1}}}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$</div>



<div id="title">

# INvestigations of massive Filaments ANd sTar formation (INFANT). I. Core Identification and Core Mass Function

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.03858-b31b1b.svg)](https://arxiv.org/abs/2404.03858)<mark>Appeared on: 2024-04-08</mark> -  _25 pages, 8 figures, accepted for ApJ_

</div>
<div id="authors">

Y. C. (程宇), et al. -- incl., <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** Filamentary structures are ubiquitously found in high-mass star-forming clouds. To investigate the relationship between filaments and star formation, we carry out the INFANT (INvestigations of massive Filaments ANd sTar formation) survey, a multi-scale, multi-wavelength survey of massive filamentary clouds with ALMA band 3/band 6 and VLA K band. In this first paper, we present the ALMA band 6 continuum observations toward a sample of 8 high-mass star forming filaments. We covered each target with approximately rectangular mosaic field of view with two 12-m array configurations, achieving an angular resolution of $\sim$ 0 $\farcs{6}$ (2700 AU at 4.5 kpc) and a continuum rms of $\sim$ 0.1 $\mjypbm$ ( $\sim$ 0.06 $\msol$ in gas mass assuming 15 K). We identify cores using the $_ getsf_$ and $_ astrodendro_$ and find the former is more robust in terms of both identification and measuring flux densities. We identify in total 183 dense cores (15--36 cores in each cloud) and classify their star formation states via outflow and warm gas tracers. The protostellar cores are statistically more massive than the prestellar cores, possibly indicating further accretion onto cores after formation of protostars. For the high-mass end ( $M_\text{core}$ $>$ 1.5 $\msun$ ) of the core mass function (CMF) we derive a power-law index of $-$ 1.15 $\pm$ 0.12 for the whole sample, and $-$ 1.70 $\pm$ 0.25 for the prestellar population. We also find a steepening trend in CMF with cloud evolution ( $-$ 0.89 $\pm$ 0.15 for the young group v.s. $-$ 1.44 $\pm$ 0.25 for the evolved group) and discuss its implication for cluster formation.

</div>

<div id="div_fig1">

<img src="tmp_2404.03858/./fmap_iden.png" alt="Fig3" width="100%"/>

**Figure 3. -** Examples of core identification results with _ getsf_ and _ astrodendro_.
The 1.3 mm image is shown in grey colorscale and black contours with levels of 0.2 $\mjypbm$$\times$(5, 10, 20, 40, 80, 160). Red ellipses show the FWHM size of cores extracted by _ getsf_, and blue contours show the core boundaries identified by _ astrodendro_.  (*fig:map_iden*)

</div>
<div id="div_fig2">

<img src="tmp_2404.03858/./fcomp_mass_update.png" alt="Fig7" width="100%"/>

**Figure 7. -** _ (a)_ 1.3 mm flux distribution for the prestellar candidate cores and protostellar cores. The dashed black line indicates the recovery fraction (or completeness level) estimated for I18310 as a representative. _ (b)_ Same as _ (a)_ but only including cores from the relatively early type clouds, defined in \autoref{sec:evolution}, i.e., I18308, I18337, I18460. _ c)_ Same as _ (a)_ but only including cores from the relatively late type clouds, defined in \autoref{sec:evolution}, i.e., I18530, I19220, I19368. _ (d)_ Mass distribution for the prestellar candidate cores and protostellar cores, where the masses are estimated using the $\ammonia$  gas temperature. The dashed lines indicate the estimated recovery fraction for I18310 as a representative. A temperature of 14 K and 19 K, i.e., the median $\ammonia$  temperature for the two populations, are adopted for the red and blue line, respectively. _ (e)_ Same as _ (d)_ but only including cores from the relatively early type clouds, i.e., I18308, I18337, I18460. _ (f)_ Same as _ (d)_ but only including cores from the relatively early type clouds, i.e., I18530, I19220, I19368.
 (*fig:comp_mass*)

</div>
<div id="div_fig3">

<img src="tmp_2404.03858/./fcmf_combine.getsf.3panel.png" alt="Fig8" width="100%"/>

**Figure 8. -** _ (a)_ CMF and prestellar CMF of the INFANT sample shown in black and blue, respectively. The core masses are calculated using the $\ammonia$  gas temperatures. The dashed lines show the best power-law fit result for the high-mass end (M $>$ 1.5 $\msun$) for the corresponding CMFs. _ (b)_ Same as _ (a)_ but only including cores from the relatively early type clouds, i.e., I18308, I18337, I18460.  _ (c)_ Same as _ (a)_ but but only including cores from the relatively late type clouds, i.e., I18530, I19220, I19368.  (*fig:cmf*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.03858"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

399  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
